## This is for storing the knowledge graph multi-dimension embeddings generated using PyKeen of text and image in FAISS index using Langchain

In [1]:
! pip install -q faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 48.0 MB/s eta 0:00:00


## Import the .csv embeddinhgs and ocnvert them into numpy array

In [22]:
import pandas as pd
import numpy as np

text_entity_embedding_path="/content/text_entity_embeddings512.csv"
text_relation_embedding_path="/content/text_relation_embeddings512.csv"
img_entity_embedding_path="/content/image_entity_embeddings512.csv"
img_relation_embedding_path="/content/image_relation_embeddings512.csv"

embeddings_img_relation=pd.read_csv(img_relation_embedding_path)
embeddings_img_entity=pd.read_csv(img_entity_embedding_path)
embeddings_img=pd.concat([embeddings_img_relation,embeddings_img_entity],axis=0)

embeddings_text_relation=pd.read_csv(text_relation_embedding_path)
embeddings_text_entity=pd.read_csv(text_entity_embedding_path)
embeddings_text=pd.concat([embeddings_text_relation,embeddings_text_entity],axis=0)

#embeddings_img.drop(columns=['url','text','Unnamed: 0'],inplace=True)
#embeddings_text.drop(columns=['prompt',"Unnamed: 0"],inplace=True)

#embeddings_img=np.load("/content/image_embeddings_train1k.npy")

embeddings_img_array=np.array(embeddings_img,dtype=np.float32) # load the image embedding
embeddings_text_array=np.array(embeddings_text,dtype=np.float32) #load the text embedding

### Storing combined image and text index

In [24]:
import faiss

dimension=512
index=faiss.IndexFlatL2(dimension)

index.add(embeddings_img_array) #add the img embedding in faiss
index.add(embeddings_text_array) # add text embedding in faiss

faiss.write_index(index, 'faiss_CLIP_txtimg_KGE.index')

### Storing individual image and text index

In [25]:
index_img=faiss.IndexFlatL2(dimension)
index_img.add(embeddings_img_array)
faiss.write_index(index_img, 'faiss_CLIP_img_KGE.index')

index_text=faiss.IndexFlatL2(dimension)
index_text.add(embeddings_text_array)
faiss.write_index(index_text, 'faiss_CLIP_text_KGE.index')

## Loading index for similarity search query

In [ ]:
index= faiss.read_index("/content/faiss_traditional_vector_embedding.index")
index
query_embedding=np.random.rand(1,512).astype(np.float32)
query_embedding
distance,indeces= index.search(query_embedding,k=5)
distance